# Creating a Vivado HLS Core


This notebook will walk through the process of creatinga a Vivado HLS core. This notebook assumes that you are familiar with Vivado HLS and have already used Vivado HLS to write, simluate, and debug a C/C++ core for hardware.


If you have not already, add Vivado HLS to your executable path. In Cygwin, you can do this by running:

``` bash
source C:/Xilinx/Vivado/2017.1/settings64.sh
```

Or on Linux: 

``` bash
source /opt/Xilinx/Vivado/2017.1/settings64.sh
```


We will be creating a simple 1-D Filter with 9 coefficients. This filter will provide two AXI-Streams, one AXI-Stream for input, and 1 AXI-Stream for output. The Filter core will also provide an AXI-Lite interface for memory-mapped writes that will configure the AXI 9 filter coefficients and HLS runtime configuration. 

This notebook assumes that you have cloned the [PYNQ-HLS repository](https://github.com/drichmond/PYNQ-HLS) to the home directory (`~`) on your computer. On our computer, this is the `/home/xilinx/` directory. 

To skip this step, run the following commands on your host computer: 

```
     cp -r ~/PYNQ-HLS/pynqhls/stream/ip/filt1d/* ~/PYNQ-HLS/tutorial/pynqhls/stream/ip/filt1d/
     make -C ~/PYNQ-HLS/pynqhls/stream/ip/filt1d
     
```


## Creating a Vivado HLS Project


We will begin by creating a Vivado HLS project. On your host computer, navigate to the `tutorial/pynqhls/stream/ip/filt1d/` folder of the PYNQ-HLS repository using your terminal:

```bash
cd ~/PYNQ-HLS/tutorial/pynqhls/stream/ip
```

In this directory we have provided a makefile that will: 

1. Create a Vivado HLS project (in the `filt1d` directory)
2. Add filt1d.cpp, and main.cpp files to the project
3. Run tests for the filt1d.cpp file (these will fail initially)
4. If the tests pass, synthesize the core.

To run the makefile, run make from your current directory:

``` bash
make
```

This will build the Vivado HLS project, but the project will fail the testbench because the method `filt1d` is not implemented. Open the project by running the command `vivado_hls` and opening the `tutorial/pynqhls/stream/ip/filt1d/filt1d` directory that was created by the makefile. This will present the following window: 

<img src="pictures/vivadohls_filt1d_splash.png" alt="Filt1d Project in Vivado HLS 2017.1 " style="width: 768px;"/>

## Writing Your Core

The next step is to implement the 1-D filter in the core. Open the file `filt1d.cpp`:

As you can see, the body of the function filt1d is blank - **this is okay**. Replace it with the implementation below: 

``` C

/* filt1d - perform a 1-dimensional filtering operation on the array INPUT, and 
return the values in OUTPUT. Each value in OUTPUT is equal to the dot product of 
the coeff array, and a corresponding range of values in INPUT.*/
void filt1d(int *INPUT, int *OUTPUT,
                int coeff[C_NUM_COEFF], unsigned int length){
/* These two pragmas define the input and output ports as AXI-Stream (axis) interfaces
with a depth of 50. An AXI-Stream is FIFO-like, non-addressed communcation*/
#pragma HLS INTERFACE axis depth=50 port=INPUT
#pragma HLS INTERFACE axis depth=50 port=OUTPUT

/* These three pragmas define a new AXI-Lite bus named CTRL for the length, 
coeff arguments, and HLS Status/Control registure (return)*/
#pragma HLS INTERFACE s_axilite port=coeff  bundle=CTRL
#pragma HLS INTERFACE s_axilite port=length bundle=CTRL
#pragma HLS INTERFACE s_axilite port=return bundle=CTRL
/* This pragma creates an array of hardware registers for the input array coeff*/
#pragma HLS ARRAY_PARTITION COMPLETE variable=coeff

/* Next, we define our signal window. We will "shift" values out of the highest index, 
and into the lowest index. We use a pragma to make window an array of hardware registers */
        int window[C_NUM_COEFF] = {0};
#pragma HLS ARRAY_PARTITION COMPLETE variable=window

/* Sum is used to compute the dot product of the current window */
        int sum;

/* The next three for loops implement the 1-D Filter operation. The outer for 
for loop reads values from the input array. The second for loop performs the shift 
operation on window, removing the oldest value, moving each value up, and putting 
the newest value in index 0. The third for loop performs the dot product operation 
between coeff and window. Pragmas are applied to maximize performance through 
pipelining and unrolling.*/
        for (unsigned int i = 0 ; i < length; i++) {
#pragma HLS PIPELINE
                for (unsigned int wi = C_NUM_COEFF-1; wi > 0; --wi){
#pragma HLS UNROLL
                        window[wi] = window[wi - 1];
                }
                window[0] = *INPUT++;
                sum = 0;
                for (unsigned int wi = 0; wi < C_NUM_COEFF; ++wi){
#pragma HLS UNROLL
                        sum += coeff[wi] * window[wi];
                }
                *OUTPUT++ = sum;
        }
}

```

Once you have filled the implementation, click the **Run C Simulation ** and then **Synthesize** button. This will produce the window shown below: 

<img src="pictures/vivadohls_filt1d_synth.png" alt="Synthesized Filt1d function in Vivado HLS 2017.1 " style="width: 768px;"/>


In the center window scroll down to view the ports, shown below: 

<img src="pictures/vivadohls_filt1d_ports.png" alt="Synthesized Filt1d function in Vivado HLS 2017.1 " style="width: 256px;"/>


The ports window shows us the interface signals for our HLS core. The port names are unimportant in this example - what matters is the protocol field. The protocol field has three types: **s_axi_lite (AXI Lite, Slave Interface)**, **axis (AXI Stream Iterface)**, and **ap_ctrl_hs (Control Signals)**. 

For best results in Vivado (and PYNQ) your core should provide **s_axi_lite**, **axis** interfaces for data transfer. These ports are automagically recognized by Vivado and can be used in the Block Diagram editor in the [Building a Bitstream](3-Building-A-Bitstream.ipynb) notebook.

**ap_ctrl_hs** signals provide clock, reset, and interrupt ports.

Finally, click the **Export RTL** button. This will bring up the following window: 

<img src="pictures/vivadohls_filt1d_export.png" alt="Exporting the filt1d function in Vivado HLS 2017.1 " style="width: 512px;"/>

Verify that the following settings are correct:

- Format Selection: **IP Catalog**
- Evaluate Generated RTL: **Verilog**

Click **OK** to continue.

When the process has completed, save, and exit Vivado HLS. Proceed to the [Building a Bitstream](3-Building-A-Bitstream.ipynb) notebook.